In [2]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
import numpy as np
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
from autogluon.common import space
from sklearn.metrics import mean_absolute_error
from typing import List, Dict
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_autogluon
importlib.reload(preprocesamiento)
importlib.reload(model_autogluon)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


# Experimento 6: 
- Autogluon: Solo redes neuronales + optimizacion de hiperparametros
    - DeepAR: Red recurrente especializada para series temporales
    - TemporalFusionTransformer: Arquitectura transformer adaptada
    - SimpleFeedForward: Red neuronal feedforward básica

- Target: mes+2

- Optimización Avanzada:
    - Búsqueda bayesiana inteligente (bayesopt)
    - 30 combinaciones diferentes de hiperparámetros
    - Rangos lógicos para parámetros clave

- Validación Robusta:
    - 5 ventanas de validación cruzada
    - Exclusión de modelos no neuronales

- Control del Proceso:
    - Logs detallados (verbosity=2)
    - Límite de tiempo opcional (3600 segundos)

- Kaggle =  0.277


In [11]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
dfg = df.groupby(['periodo', 'product_id']).agg({'target': 'sum'}).reset_index()
dfg['timestamp'] = pd.to_datetime(dfg['periodo'].astype(str), format='%Y%m')
dfg.rename(columns={'product_id': 'item_id'}, inplace=True)

# Filtrar productos
productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
dfg = dfg[dfg['item_id'].isin(productos_ok['product_id'].unique())]

# Convertir a TimeSeriesDataFrame
data = TimeSeriesDataFrame.from_data_frame(
    dfg,
    id_column="item_id",
    timestamp_column="timestamp"
)

In [12]:
# 2. Configurar y entrenar el predictor solo con modelos neuronales
predictor = TimeSeriesPredictor(
    target='target',
    prediction_length=1,  # Predecir un paso adelante (mes+2)
    freq="M",
    eval_metric="MAPE"
).fit(
    train_data=data,
    # Especificamos solo modelos neuronales
    hyperparameters={
        'DeepAR': {},
        'TemporalFusionTransformer': {},
        'SimpleFeedForward': {}
    },
    # Excluimos modelos no neuronales
    excluded_model_types=['ARIMA', 'ETS', 'Theta'],
    num_val_windows=5,
    verbosity=2
)

# 3. Evaluar y hacer predicciones
performance = predictor.evaluate(data)
print(f"\nDesempeño del modelo: {performance}")

predictions = predictor.predict(data)
print("\nPredicciones generadas:")
print(predictions.head())

Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\notebooks\entregable\AutogluonModels\ag-20250629_134010'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       2.46 GB / 15.89 GB (15.5%)
Disk Space Avail:   413.67 GB / 893.49 GB (46.3%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'excluded_model_types': ['ARIMA', 'ETS', 'Theta'],
 'freq': 'ME',
 'hyperparameters': {'DeepAR': {},
                     'SimpleFeedForward': {},
                     'TemporalFusionTransformer': {}},
 'known_covariates_names': [],
 'num_val_windows': 5,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_ev


Desempeño del modelo: {'MAPE': nan}


data with frequency 'MS' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: TemporalFusionTransformer



Predicciones generadas:
                           mean         0.1         0.2          0.3  \
item_id timestamp                                                      
20001   2020-01-31  1341.511475  823.683533  925.015564  1085.486206   
20002   2020-01-31  1015.737305  534.233826  632.800842   789.344604   
20003   2020-01-31   814.567871  409.315521  488.814697   613.297180   
20004   2020-01-31   618.369019  299.359558  364.609222   461.017120   
20005   2020-01-31   577.933289  270.242493  335.133575   425.597534   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
20001   2020-01-31  1214.482422  1341.511475  1464.198853  1623.361328   
20002   2020-01-31   906.329956  1015.737305  1130.787842  1293.329468   
20003   2020-01-31   712.641174   814.567871   908.631897  1035.952637   
20004   2020-01-31   538.686951   618.369019   691.794373   791.926880   
20005   2020-01-31   499.8

In [16]:
predictions_copy = predictions.copy()
predictions_copy = predictions_copy.reset_index()
predictions_copy.rename(columns={'mean': 'tn', 'item_id': 'product_id'}, inplace=True)
predictions_copy = predictions_copy[['product_id','tn']]
output_path = "./outputs/predicciones_exp_06v4_autogluon_v1_original.csv"
predictions_copy.to_csv(output_path, index=False)

##### Opcion 2

In [44]:
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
from autogluon.common import space

def load_and_prepare_data():
    """Carga y transforma los datos para predecir ventas en mes+2."""
    # Leer datos crudos
    df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
    df['cat1'].fillna("Ninguno", inplace=True)
    df['cat2'].fillna("Ninguno", inplace=True)
    df['cat3'].fillna("Ninguno", inplace=True)
    df['brand'].fillna("Ninguno", inplace=True)
    

    
    # Convertir periodo a datetime y renombrar columnas
    df['timestamp'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')
    df.rename(columns={
        'tn': 'current_sales',  # Covariable: Ventas actuales
        'product_id': 'item_id'
    }, inplace=True)

    # Filtrar productos relevantes
    productos_ok = pd.read_csv('../../data/raw/product_id_apredecir201912.csv', sep=',')
    df = df[df['item_id'].isin(productos_ok['product_id'].unique())]
    
    # Eliminar filas donde target es NA (las últimas 2 de cada serie)
    df = df.dropna(subset=['target'])
    
    return df

def get_neural_config():
    """Define hiperparámetros para modelos neuronales."""
    return {
        'DeepAR': {
            'epochs': space.Int(30, 100),
            'num_cells': space.Categorical([64, 128, 256]),
            'context_length': space.Int(12, 24),
            'dropout_rate': space.Real(0.1, 0.3),
            'learning_rate': space.Real(1e-4, 1e-2, log=True)
        },
        'TemporalFusionTransformer': {
            'epochs': space.Int(20, 50),
            'hidden_dim': space.Categorical([64, 128]),
            'dropout': space.Real(0.1, 0.3),
            'attention_head_size': space.Int(2, 4)
        }
    }

def train_model(data):
    """Entrena el predictor con optimización avanzada."""
    predictor = TimeSeriesPredictor(
        target='target',
        prediction_length=1,
        freq="M",
        eval_metric="MAPE",
        quantile_levels=[0.1, 0.5, 0.9]
    )
    
    predictor.fit(
        train_data=data,
        hyperparameters={
        'DeepAR': {},
        'TemporalFusionTransformer': {},
        'SimpleFeedForward': {}
        },
        # Excluimos modelos no neuronales
        excluded_model_types=['ARIMA', 'ETS', 'Theta'],
        # hyperparameter_tune_kwargs={
        #     'num_trials': 30,
        #     'scheduler': 'local',
        #     'searcher': 'bayes'
        # },
        num_val_windows=5,
        verbosity=2
    )
    return predictor

def main_v2():
    # Cargar y preparar datos
    df = load_and_prepare_data()
    
    # Preparar covariables conocidas
    covariates = df[['item_id', 'timestamp', 'current_sales']].copy()
    
    # Convertir a TimeSeriesDataFrame
    data = TimeSeriesDataFrame.from_data_frame(
        df[['item_id', 'timestamp', 'target']],
        id_column="item_id",
        timestamp_column="timestamp"
    )
    
    # Añadir covariables manualmente
    # data.static_features = df[['cat1', 'cat2', 'cat3', 'brand']]
    data.past_covariates = covariates
    
    # Entrenar modelo
    predictor = train_model(data)
    
    # Evaluar
    performance = predictor.evaluate(data)
    print(f"\n🔍 Resultados de evaluación:")
    print(f"MAPE: {performance['MAPE']:.2%}")
    
    
    # Predecir (necesitarías proporcionar futuras covariables en producción)
    predictions = predictor.predict(data)
    print("\n🔮 Predicciones para mes+2:")
    print(predictions.head())
    
    # Guardar modelo
    # predictor.save("autogluon_mes+2_model")
    print("\n💾 Modelo guardado en 'autogluon_mes+2_model'")

#### Main execution
main_v2()

No path specified. Models will be saved in: "AutogluonModels\ag-20250629_184335"
Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\notebooks\entregable\AutogluonModels\ag-20250629_184335'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          4
GPU Count:          0
Memory Avail:       2.07 GB / 15.89 GB (13.0%)
Disk Space Avail:   413.59 GB / 893.49 GB (46.3%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAPE,
 'excluded_model_types': ['ARIMA', 'ETS', 'Theta'],
 'freq': 'ME',
 'hyperparameters': {'DeepAR': {},
                     'SimpleFeedForward': {},
                     'TemporalFusionTransformer': {}},
 'known_covariates_names': [],
 'num_val_windows': 5,
 'prediction_length': 1,
 'quantile_levels

In [42]:
# Predecir (necesitarías proporcionar futuras covariables en producción)
predictions = predictor.predict(data)
print("\n🔮 Predicciones para mes+2:")
print(predictions.head())

data with frequency 'MS' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: TemporalFusionTransformer


In [ ]:
predictions_copy = predictions.copy()
predictions_copy = predictions_copy.reset_index()
predictions_copy.rename(columns={'mean': 'tn', 'item_id': 'product_id'}, inplace=True)
predictions_copy = predictions_copy[['product_id','tn']]
output_path = "./outputs/predicciones_exp_06v4_autogluon_v1_original.csv"
predictions_copy.to_csv(output_path, index=False)

,,mean,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
item_id,timestamp,,,,,,,,,,
20001,2020-01-31,1341.511475,823.683533,925.015564,1085.486206,1214.482422,1341.511475,1464.198853,1623.361328,1817.171265,1987.480835
20002,2020-01-31,1015.737305,534.233826,632.800842,789.344604,906.329956,1015.737305,1130.787842,1293.329468,1469.710083,1626.574951
20003,2020-01-31,814.567871,409.315521,488.814697,613.297180,712.641174,814.567871,908.631897,1035.952637,1188.012695,1324.846191
20004,2020-01-31,618.369019,299.359558,364.609222,461.017120,538.686951,618.369019,691.794373,791.926880,911.070190,1019.771729
20005,2020-01-31,577.933289,270.242493,335.133575,425.597534,499.880157,577.933289,648.736572,743.375427,858.381592,965.774536
...,...,...,...,...,...,...,...,...,...,...,...
20962,2020-01-31,1.781170,0.106227,0.709827,1.001992,1.667561,1.781170,2.223029,2.749121,3.341600,3.908631
20975,2020-01-31,1.547632,0.082958,0.610555,0.866282,1.448722,1.547632,1.932698,2.391508,2.909231,3.405061
20995,2020-01-31,1.488675,0.061618,0.575264,0.824913,1.393176,1.488675,1.861265,2.305810,2.809428,3.292396
